# 1. Read in all the XML reports

Description: Cycle through all of the xml reports and convert to 3 datasets

- lungcaddata: NOT USED
- scan_info_data: holds scan level info, including spacing, origiin etc.
- nodule_info_dtaa: holds raw Veolity generated nodule data


In [1]:

import pandas as pd
from pathlib import Path
import xml.etree.ElementTree as ET


def recursive_parse(element, parent_key=''):
    items = {}
    for child in element:
        key = f"{parent_key}.{child.tag}" if parent_key else child.tag
        if len(child):
            items.update(recursive_parse(child, key))
        else:
            items[key] = child.text
    return items

def parse_xml(xml_file):
    

    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extracting LUNGCAD data
    for lungcad in root.findall('LungCAD'):
        lungcad_data = recursive_parse(lungcad)

    # Extracting scan_info data
    scan_info_data = recursive_parse(root.findall('ImageInfo'))

    nodules_data = []
    # Extracting nodules data
    for nodule in root.findall('*Finding'):
        nodules_data.append(recursive_parse(nodule))

    return lungcad_data, scan_info_data, nodules_data


xml_scan_data = []
xml_nodule_data = []

for xml_file in Path('Veolity/LSUT-tranche-2').rglob('*.xml'):
    if xml_file.suffix == '.xml':

        _, scan_info_data, nodules_data = parse_xml(xml_file)


        xml_scan_data.append(scan_info_data)

        for nodule in nodules_data:
            nodule['PatientUID'] = scan_info_data['ImageInfo.PatientUID']
            nodule['SeriesUID'] = scan_info_data['ImageInfo.SeriesUID']
        xml_nodule_data.extend(nodules_data)


xml_scan_data = pd.DataFrame(xml_scan_data)
display(xml_scan_data.head())

xml_nodule_data = pd.DataFrame(xml_nodule_data)
xml_nodule_data['X'] = xml_nodule_data['X'].astype(float)
xml_nodule_data['Y'] = xml_nodule_data['Y'].astype(float)
xml_nodule_data['Z'] = xml_nodule_data['Z'].astype(float)
xml_nodule_data['Probability'] = xml_nodule_data['Probability'].astype(float)
xml_nodule_data['Volume_mm3'] = xml_nodule_data['Diameter_mm'].astype(float)
xml_nodule_data['Diameter_mm'] = xml_nodule_data['Diameter_mm'].astype(float)
display(xml_nodule_data.head())


,ImageInfo.Dimensions.dimX,ImageInfo.Dimensions.dimY,ImageInfo.Dimensions.dimZ,ImageInfo.VoxelSize.voxelSizeX,ImageInfo.VoxelSize.voxelSizeY,ImageInfo.VoxelSize.voxelSizeZ,ImageInfo.Origin.originX,ImageInfo.Origin.originY,ImageInfo.Origin.originZ,ImageInfo.Orientation,ImageInfo.PatientName,ImageInfo.PatientUID,ImageInfo.StudyUID,ImageInfo.SeriesUID
0,512,512,301,0.625,0.625,0.799927,-160,-160,-992.5,"1, 0, 0, 0, 1, 0, 0, 0, 1",UCLH_99789879,UCLH_99789879,1.3.6.1.4.1.34261.20.6525579266131913678813993...,1.3.6.1.4.1.34261.20.3827951232666181738346682...
1,512,512,372,0.698,0.698,0.800049,-187.932,-178.557,1399.7,"1, 0, 0, 0, 1, 0, 0, 0, 1",UCLH_88328027,UCLH_88328027,1.3.6.1.4.1.34261.20.6792713814436424459711141...,1.3.6.1.4.1.34261.20.7927556837918446841634615...
2,512,512,406,0.689,0.689,0.800049,-177.78,-176.218,1607,"1, 0, 0, 0, 1, 0, 0, 0, 1",UCLH_99045724,UCLH_99045724,1.3.6.1.4.1.34261.20.6663528461261714936272711...,1.3.6.1.4.1.34261.20.1472567476738636383795337...
3,512,512,376,0.625,0.625,0.799927,-178.125,-160.156,-1052.5,"1, 0, 0, 0, 1, 0, 0, 0, 1",UCLH_87656994,UCLH_87656994,1.3.6.1.4.1.34261.20.2791369571227957663788352...,1.3.6.1.4.1.34261.20.9569932349493313461366419...
4,512,512,326,0.781,0.781,0.800049,-199.609,-199.609,1698,"1, 0, 0, 0, 1, 0, 0, 0, 1",UCLH_99493491,UCLH_99493491,1.3.6.1.4.1.34261.20.2295892354828912716998471...,1.3.6.1.4.1.34261.20.9166792928438334261496677...


,ID,X,Y,Z,Probability,Volume_mm3,Diameter_mm,PatientUID,SeriesUID
0,16,60.6139,101.4490,1727.8,3719.410,6.08395,6.08395,UCLH_99045724,1.3.6.1.4.1.34261.20.1472567476738636383795337...
1,10,30.6250,90.4688,-866.9,-884.444,5.40518,5.40518,UCLH_87656994,1.3.6.1.4.1.34261.20.9569932349493313461366419...
2,16,-47.5000,80.4688,-884.5,-947.946,5.12683,5.12683,UCLH_87656994,1.3.6.1.4.1.34261.20.9569932349493313461366419...
3,30,88.7500,-43.2812,-926.9,-916.489,6.34373,6.34373,UCLH_87656994,1.3.6.1.4.1.34261.20.9569932349493313461366419...
4,37,-106.2500,-67.0312,-943.7,972.368,4.10075,4.10075,UCLH_87656994,1.3.6.1.4.1.34261.20.9569932349493313461366419...


# 2. Get Scan Metadata

For each scan there are multiple series, for consistency with SUMMIT we need to only use the soft tissue Veolity report.

In [2]:
scan_metadata = pd.read_csv('/Users/john/Projects/ScansTransfer/LSUT/dicom_metadata_with_recon.csv')
scan_patient_uids = set(scan_metadata['SubjectID'].unique())

display(scan_metadata.head())

,SeriesInstanceUID,SubjectID,SeriesDescription,NumberOfFiles,filesby10,Reconstruction
0,1.3.6.1.4.1.34261.20.9893711945195858342155588...,UCLH_00134949,2.0,2,0,DO_NOT_USE
1,1.3.6.1.4.1.34261.20.3438257846713158166155226...,UCLH_00134949,Lung 1.0 hom-1082/LDLS,326,32,Lung
2,1.3.6.1.4.1.34261.20.4941623947498558212956748...,UCLH_00134949,5 hom-1082/LDLS,2,0,DO_NOT_USE
3,1.3.6.1.4.1.34261.20.8746993336615974177722837...,UCLH_00134949,Body 1.0 hom-1082/LDLS,326,32,Soft
4,1.3.6.1.4.1.34261.20.4727291588139957739148357...,UCLH_00239233,2.0,2,0,DO_NOT_USE


In [10]:
# Run some quick checks on the numbers of xml reports we have and how many are missing
soft_metadata = scan_metadata.query('Reconstruction == "Soft"')
soft_patient_uids = set(soft_metadata['SubjectID'].unique())

print(f"Number of patients with soft reconstructions: {len(soft_patient_uids)}")

xml_patient_uids = set(xml_scan_data['ImageInfo.PatientUID'].unique())
print(f"Number of patients with xml reports: {len(xml_patient_uids)}")

missing_patient_uids = soft_patient_uids - xml_patient_uids
print(f"Number of patients with soft reconstructions but no xml reports: {len(missing_patient_uids)}")

subset_patient_uids = soft_patient_uids.intersection(xml_patient_uids) - set(xml_nodule_data['PatientUID'].unique())
print(f"Number of patients with soft reconstructions and xml reports but no nodules: {len(subset_patient_uids)}")
with open('tranche2_soft_recon_patients_with_no_nodules.txt', 'w') as f:
    for patient_uid in subset_patient_uids:
        f.write(f"{patient_uid}\n")


Number of patients with soft reconstructions: 757
Number of patients with xml reports: 146
Number of patients with soft reconstructions but no xml reports: 611
Number of patients with soft reconstructions and xml reports but no nodules: 33


# 3. Isolate the Soft Tissue Generated Nodules

In [4]:
soft_series_uids = set(soft_metadata['SeriesInstanceUID'].unique())
soft_xml_nodule_data = xml_nodule_data.query('SeriesUID in @soft_series_uids')

print(f"Number of scans in XML:", len(xml_patient_uids))
print(f"Number of nodules in XML:", len(xml_nodule_data))

scan_ids_with_nodules = set(soft_xml_nodule_data['PatientUID'].unique())

Number of scans in XML: 146
Number of nodules in XML: 699


# 4. Make sure that the scan exists on disk

In [5]:
lung_listings = (
    pd.read_csv('lung_listings.txt', header=None, names=['nifti_path'])
    .assign(patient_uid=lambda x: x['nifti_path'].str.split('/').str[-2])
)

lung_patient_uids = set(lung_listings['patient_uid'].unique())
soft_xml_patient_uids = set(soft_xml_nodule_data['PatientUID'].unique())

missing_in_lung_listings = soft_xml_patient_uids - lung_patient_uids
missing_in_soft_xml = lung_patient_uids - soft_xml_patient_uids

print(f"Number of patients in lung_listings:", len(lung_patient_uids))
print(f"Number of patients in soft_xml_nodule_data:", len(soft_xml_patient_uids))
print(f"IDs in soft_xml_nodule_data but not in lung_listings:", len(missing_in_lung_listings))
print(f"IDs in lung_listings but not in soft_xml_nodule_data:", len(missing_in_soft_xml))

Number of patients in lung_listings: 731
Number of patients in soft_xml_nodule_data: 96
IDs in soft_xml_nodule_data but not in lung_listings: 2
IDs in lung_listings but not in soft_xml_nodule_data: 637


# 5. Checks on scan / nodule data

In [15]:
import nibabel as nib
import numpy as np
import subprocess


def copy_and_mask(mask_id, source_path, destination_path):


    if Path(f"{source_path}/box-masks/{mask_id}").exists():
        print(f"Box masks for {mask_id} already exists")
    else:
        subprocess.run(
            [
                "scp", 
                "-P", 
                "2222", 
                "-r", 
                f"jmccabe@localhost:{source_path}/box-masks/{mask_id}",
                f"{destination_path}/box-masks/."
            ],
            check=True
        )

    if Path(f"{source_path}/detection/{mask_id}").exists():
        print(f"Detection masks for {mask_id} already exists")
    else:
        subprocess.run(
            [
                "scp", 
                "-P", 
                "2222", 
                "-r", 
                f"jmccabe@localhost:{source_path}/detection/{mask_id}",
                f"{destination_path}/detection/."
            ],
            check=True
        )

    return True

source_path = "/cluster/project0/lung-triage/lsut"
destination_path = "../../cache/sota/lsut"

# for mask_id in ['UCLH_00489272','UCLH_00239233','UCLH_01212990','UCLH_00948384','UCLH_01133076','UCLH_00134949']:
#     _ = copy_and_mask(mask_id, source_path, destination_path)


# 7. Run comparisons against annotations

1. Check how many scans with nodules have been processed and how many are left to be processed
2. Check that the number of nodules marries up with the number of boxes
3. Convert the boxes to row, col, slice (for initial resolution)
4. Check box exists for recorded nodules, convert slices-nod_slice in real world co-ordinates

In [7]:
import numpy as np
def pixel_to_real_world(offset, spacing, pixel_value):
    return offset + pixel_value * spacing

annotations = pd.read_csv('annotations.csv')
metaio_metadata = pd.read_csv('lung_metadata.csv').assign(scan_id=lambda x: x['scan_id'].str.replace('.mhd', ''))

annotations = pd.merge(
    metaio_metadata,
    annotations,
    left_on='scan_id',
    right_on='ScananonID',
    how='left'
)

annotations['Nod1_floc'] = annotations.apply(
    lambda row: row['slices'] - row['Nod1_loc'] if pd.notnull(row['Nod1_loc']) else None, axis=1
)

annotations['Nod2_floc'] = annotations.apply(
    lambda row: row['slices'] - row['Nod2_loc'] if pd.notnull(row['Nod2_loc']) else None, axis=1
)
    
annotations['Nod1_real_world'] = annotations.apply(
    lambda row: pixel_to_real_world(row['z-offset'], row['z-spacing'], row['Nod1_floc']) if pd.notnull(row['Nod1_floc']) else (None), axis=1
)

annotations['Nod2_real_world'] = annotations.apply(
    lambda row: pixel_to_real_world(row['z-offset'], row['z-spacing'], row['Nod2_floc']) if pd.notnull(row['Nod2_floc']) else (None), axis=1
)


annotations['Reader'] = np.random.choice([1, 2], size=len(annotations), p=[0.5, 0.5])

print('Total number of annotations:', annotations.shape[0])
print('Total number of scans with soft reconstructions:', soft_metadata.shape[0])
print('Total number of scans saved to nifti (Useable):', lung_listings.shape[0])

with_nodule_annotation_uids = set(annotations.query('Total_no_nods > 0')['ScananonID'])
display(annotations.query('ScananonID in @with_nodule_annotation_uids')['clinic_ethnicity'].value_counts())
display(annotations.query('ScananonID in @with_nodule_annotation_uids')['clinic_gender'].value_counts())

no_nodule_annotation_uids = set(annotations.query('Total_no_nods == 0')['ScananonID'])
xml_nodule_patient_uids = set(xml_nodule_data['PatientUID'])

print('No nodules, with xml report:', len(no_nodule_annotation_uids.intersection(xml_nodule_patient_uids)))
double_negative_uids = list(no_nodule_annotation_uids.intersection(xml_patient_uids) - xml_nodule_patient_uids)

print('Whole sample')
display(pd.DataFrame(annotations['clinic_ethnicity'].value_counts()))
display(pd.DataFrame(annotations['clinic_gender'].value_counts()))

print('No nodules, with xml report but no xml nodules recorded:', len(no_nodule_annotation_uids.intersection(xml_patient_uids) - xml_nodule_patient_uids))
display(pd.DataFrame(annotations.query('ScananonID in @double_negative_uids')['clinic_ethnicity'].value_counts()))
display(pd.DataFrame(annotations.query('ScananonID in @double_negative_uids')['clinic_gender'].value_counts()))

print('With nodules:', len(with_nodule_annotation_uids))
display(pd.DataFrame(annotations.query('ScananonID in @with_nodule_annotation_uids')['clinic_ethnicity'].value_counts()))
display(pd.DataFrame(annotations.query('ScananonID in @with_nodule_annotation_uids')['clinic_gender'].value_counts()))

nifti_patient_uids = set(lung_listings['patient_uid'])
usable_patient_uids = nifti_patient_uids.intersection(xml_patient_uids)
annotations = annotations.query('ScananonID in @usable_patient_uids')

print('*' * 50)
print('Total number of nifti with xml:', len(usable_patient_uids))


print('-Usable without nodules (Total_no_nods):', annotations.query('Total_no_nods == 0').shape[0])
print('-Usable with nodules (Total_no_nods):', annotations.query('Total_no_nods > 0').shape[0])
print('--Useable with nodules but Nod1_loc is Null:', annotations.query('Total_no_nods > 0 and Nod1_loc.isnull()').shape[0])

scans_with_xml_scan_info = set(annotations.query('Total_no_nods > 0')['ScananonID']).intersection(xml_scan_data['ImageInfo.PatientUID'])
print('--Useable with nodules with xml scan info:', len(scans_with_xml_scan_info))

scans_with_xml_nodules = set(annotations.query('Total_no_nods > 0')['ScananonID']).intersection(xml_nodule_data['PatientUID'])
print('--Useable with nodules with xml nodules:', len(scans_with_xml_nodules))

Total number of annotations: 732
Total number of scans with soft reconstructions: 757
Total number of scans saved to nifti (Useable): 731


/var/folders/59/cd3dzx7560qgtrwc99hm0j2w0000gp/T/ipykernel_69020/2595400772.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  metaio_metadata = pd.read_csv('lung_metadata.csv').assign(scan_id=lambda x: x['scan_id'].str.replace('.mhd', ''))


White                                       133
Black/ African/ Caribbean/ Black British     13
Other ethnic group                            9
Mixed or multiple ethnic groups               2
Asian or Asian British                        1
Name: clinic_ethnicity, dtype: int64

Male      91
Female    67
Name: clinic_gender, dtype: int64

No nodules, with xml report: 73
Whole sample


...

,clinic_gender
Male,22
Female,10


With nodules: 158


,clinic_ethnicity
White,133
Black/ African/ Caribbean/ Black British,13
Other ethnic group,9
Mixed or multiple ethnic groups,2
Asian or Asian British,1


,clinic_gender
Male,91
Female,67


**************************************************
Total number of nifti with xml: 144
-Usable without nodules (Total_no_nods): 105
-Usable with nodules (Total_no_nods): 39
--Useable with nodules but Nod1_loc is Null: 8
--Useable with nodules with xml scan info: 39
--Useable with nodules with xml nodules: 38


In [8]:

np.random.seed(42)  # For reproducibility


reader1_nodule_samples = annotations.query('Total_no_nods > 0 and Reader == 1')
reader2_nodule_samples = annotations.query('Total_no_nods > 0 and Reader == 2')


print()
print('Reader 1 samples with nodules:',len(reader1_nodule_samples))
print('Reader 1 samples without nodule annotations:',len(reader1_nodule_samples.query('Nod1_loc.isnull()')))
print('Reader 1 samples with nodules but no xml nodules:',len(reader1_nodule_samples.query('ScananonID not in @scans_with_xml_nodules')))


print()
print('Reader 2 samples with nodules:',len(reader2_nodule_samples))
print('Reader 2 samples without nodule annotations:',len(reader2_nodule_samples.query('Nod1_loc.isnull()')))
print('Reader 2 samples with nodules but no xml nodules:',len(reader2_nodule_samples.query('ScananonID not in @scans_with_xml_nodules')))

print()

display(annotations.clinic_ethnicity.value_counts())
display(pd.concat([reader1_nodule_samples,reader2_nodule_samples]).clinic_ethnicity.value_counts())
display(pd.concat([reader1_nodule_samples,reader2_nodule_samples]).clinic_gender.value_counts())


Reader 1 samples with nodules: 24
Reader 1 samples without nodule annotations: 5
Reader 1 samples with nodules but no xml nodules: 0

Reader 2 samples with nodules: 15
Reader 2 samples without nodule annotations: 3
Reader 2 samples with nodules but no xml nodules: 1



White                                       122
Black/ African/ Caribbean/ Black British     13
Other ethnic group                            7
Mixed or multiple ethnic groups               2
Name: clinic_ethnicity, dtype: int64

White                                       34
Other ethnic group                           2
Black/ African/ Caribbean/ Black British     2
Mixed or multiple ethnic groups              1
Name: clinic_ethnicity, dtype: int64

Male      23
Female    16
Name: clinic_gender, dtype: int64

In [9]:

import json
def generate_json_markup(nodule_data):

    slicer_markup = {
    "@schema": "https://raw.githubusercontent.com/slicer/slicer/master/Modules/Loadable/Markups/Resources/Schema/markups-schema-v1.0.3.json#",
    "markups": [
        {
            "type": "Fiducial",
            "coordinateSystem": "LPS",
            "coordinateUnits": "mm",
            "locked": False,
            "fixedNumberOfControlPoints": False,
            "labelFormat": "%N-%d",
            "lastUsedControlPointNumber": 1,
            "controlPoints": [
                {
                    "id": f"{nodule.ID}",
                    "label": f"F-{i}",
                    "description": f"{nodule.ID}",
                    "associatedNodeID": "vtkMRMLScalarVolumeNode1",
                    "position": [
                        nodule.X,
                        nodule.Y,
                        nodule.Z
                    ],
                    "orientation": [
                        -1.0,
                        -0.0,
                        -0.0,
                        -0.0,
                        -1.0,
                        -0.0,
                        0.0,
                        0.0,
                        1.0
                    ],
                    "selected": True,
                    "locked": False,
                    "visibility": True,
                    "positionStatus": "defined"
                }
                for i, nodule in enumerate(nodule_data.itertuples())
            ],
            "measurements": [],
            "display": {
                "visibility": True,
                "opacity": 1.0,
                "color": [
                    0.4,
                    1.0,
                    1.0
                ],
                "selectedColor": [
                    1.0,
                    0.5000076295109483,
                    0.5000076295109483
                ],
                "activeColor": [
                    0.4,
                    1.0,
                    0.0
                ],
                "propertiesLabelVisibility": False,
                "pointLabelsVisibility": True,
                "textScale": 3.0,
                "glyphType": "Sphere3D",
                "glyphScale": 3.0,
                "glyphSize": 5.0,
                "useGlyphScale": True,
                "sliceProjection": False,
                "sliceProjectionUseFiducialColor": True,
                "sliceProjectionOutlinedBehindSlicePlane": False,
                "sliceProjectionColor": [
                    1.0,
                    1.0,
                    1.0
                ],
                "sliceProjectionOpacity": 0.6,
                "lineThickness": 0.2,
                "lineColorFadingStart": 1.0,
                "lineColorFadingEnd": 10.0,
                "lineColorFadingSaturation": 1.0,
                "lineColorFadingHueOffset": 0.0,
                "handlesInteractive": False,
                "translationHandleVisibility": True,
                "rotationHandleVisibility": True,
                "scaleHandleVisibility": False,
                "interactionHandleScale": 3.0,
                "snapMode": "toVisibleSurface"
            }
        }
    ]
}   
    return slicer_markup

for patient_uid in xml_nodule_data['PatientUID'].unique():
    markup_json = generate_json_markup(xml_nodule_data.query(f"PatientUID == '{patient_uid}'"))
    reader = 'reader1' if patient_uid in reader1_nodule_samples['ScananonID'].unique() else 'reader2'
    json.dump(markup_json, open(f'RadiologistReview/tranche2/{reader}/{patient_uid}.json', 'w'), indent=4)
    